# MEC_51053_EP — Numerical methods in (solid) mechanics — Martin Genet

# L9 — Partial differential equations (Elastodynamics initial/boundary value problem) — The finite element method + Integration schemes 

# E9.1 — Vibrations — Matthias Rambausek & Martin Genet

As usual, let us start with *some* imports…

In [ ]:
# For better printing within jupyter cells (otherwise only the last variable is printed)
import IPython
IPython.core.interactiveshell.InteractiveShell.ast_node_interactivity = "all"

# Standard python libraries
import math
import os
import time

# Computing libraries
import numpy

# Meshing libraries
import gmsh
import meshio

# FEniCS
import dolfin # dolfin if the main interface to FEniCS
dolfin.parameters["form_compiler"]["cpp_optimize"] = True
dolfin.parameters["form_compiler"]["optimize"] = True

# VTK and visualization
import itkwidgets
import myVTKPythonLibrary as myvtk

# LIB552 python library
import LIB552 as lib

## Problem

In this exercise we compute and analyze the vibration frequencies and modes of a thin disc (radius $R$, thickness $Z$), made of an isotropic linear elastic material (mass density $\rho$, Young modulus $E$, Poisson ratio $\nu$).
In principle one would use a (2D) plate model for such a problem, but here we will model it in 3D.

Note that this notebook is inspired by an example from [Jeremy Bleyer's COMET website](https://comet-fenics.readthedocs.io/en/latest).

### Parameters

We set some numerical values for the physical parameters.

In [ ]:
# Geometry
R = 1.
Z = 0.1

# Material
rho   = 1.
E     = 1.
nu    = 0.3
lmbda = E * nu / (1 + nu) / (1 - 2 * nu)
mu    = E / 2 / (1 + nu)

## Finite element resolution

**Q1.
Derive the weak formulation of the vibration problem, in terms of the vibration frequency $\Omega$ and the vibration mode $\underline{U}$.**

Hint:
 * Simply inject a vibration solution of the form $\underline{u}\left(\underline{x},t\right) = \underline{U}\left(\underline{x}\right)\sin\left(\Omega t\right)$ into the elasto-dynamic equations, and follow the Galerkin procedure to turn the system of local equations into a single global variational equation of the form $\underline{U},\Omega ~/~ b_K\left(\underline{U}, \underline{U^*}\right) = \Omega^2~b_M\left(\underline{U}, \underline{U^*}\right) ~ \forall \underline{U^*}$, where $b_K$ and $b_M$ are symmetric coercive bilinear forms, both to be expressed in terms of the problem parameters.

**Q2.
Derive the associated discrete generalized eigenproblem, in terms of the vibration frequency $\Omega$ and the discrete vibration mode $\underline{\mathbb{U}}$.**

Hint:
 * Simply inject the discretization $\underline{U}\left(\underline{x}\right) \approx {}^{t}{\underline{\underline{\mathbb{N}}}}\left(\underline{x}\right) \cdot \underline{\mathbb{U}}$ into the weak formulation.

### Parameters

We define the computation parameters.

In [ ]:
# Number of elements in the radius of the disc
N_R = 10

# Number of elements in the thickness of the disc
N_Z = 10

# Target number of computed modes
N_modes = 16

### Mesh

We build the mesh, for instance using constructive geometry.
In order to have a fine control on the element size (notably, different refinements in the plane in the thickness), we first generate a disc, and then extrude it.

**Q3.
Complete and execute the following code.**

Hints:
* `factory.addDisk(xc, yc, zc, rx, ry)` creates a (2D) ellipsoid centered in [xc,yc,zc], aligned with the (x,y) plane, with radii [rx,ry], and returns its tag.
* `factory.extrude(dimTags=[(2,s1)], dx=dx, dy=dy, dz=dz, numElements=[n])[1][1]` extrudes the (2D) surface tagged `s1` by the vector [dx, dy, dz], creating n layers of elements, and returns the tag of the generated (3D) volume.

In [ ]:
## Initialization
gmsh.initialize()
gmsh.clear()

## Geometry
factory = gmsh.model.occ
disk_surf_tag = ### YOUR CODE HERE ###
disk_volu_tag = ### YOUR CODE HERE ###

# Synchronization, cf., e.g., https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/tutorial/python/t16.py
factory.synchronize()

# In order to only save nodes and elements of the final mesh
# (i.e., not the construction nodes and elements—remember that 
# unstructured meshers will first mesh the curves, then the
# surfaces and the volumes, cf. L5.2), we declare it as a
# "physical" entity.
disk_phys_tag = gmsh.model.addPhysicalGroup(dim=3, tags=[disk_volu_tag])

## Mesh
mesh_gmsh = gmsh.model.mesh

# Characteristic size, cf., e.g., https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/tutorial/python/t16.py
mesh_gmsh.setSize(gmsh.model.getEntities(0), R/N_R)

# Mesh generation
mesh_gmsh.generate(dim=3)

# In order to visualize the mesh and perform finite element computation using
# FEniCS, we need to convert the mesh from the GMSH format to the VTK & FEniCS
# formats. Since there is no direct converter between these formats, we do
# that here by writing the mesh to the disc in VTK format using GMSH, which
# we can then read in various formats later on.
gmsh.write("E9.1-Vibrations-mesh.vtk")

# Finalization
gmsh.finalize()

Let us visualize the mesh, using [itkwidgets](https://github.com/InsightSoftwareConsortium/itkwidgets).

In [ ]:
mesh_vtk = myvtk.readUGrid("E9.1-Vibrations-mesh.vtk")
itkwidgets.view(geometries=mesh_vtk)

Let us now convert the mesh into the FEniCS format, using [meshio](https://github.com/nschloe/meshio).

In [ ]:
mesh_meshio = meshio.read("E9.1-Vibrations-mesh.vtk")
meshio.write("E9.1-Vibrations-mesh.xdmf", mesh_meshio)

mesh = dolfin.Mesh()
dolfin.XDMFFile("E9.1-Vibrations-mesh.xdmf").read(mesh)

### Finite element and function space

We now define the interpolation space, starting with a finite element structure, very similar (though much more complete) to the one of LIB552 used in previous notebooks.

In [ ]:
fe = dolfin.VectorElement(
    family="Lagrange",
    cell=mesh.ufl_cell(), # triangle in 2D, tetrahedron in 3D
    degree=1)
fe

The function space structure is based on the mesh and the finite element, it contains notably the dof manager.

In [ ]:
fs = dolfin.FunctionSpace(mesh, fe)
fs

# Total number of degrees of freedom
fs.dim()

### Finite element matrices

Let us start by assembling the mass matrix.
Do do so, we define the bilinear form associated to the matrix.
It must be bilinear in the solution function (called trial function) and the test function.

In [ ]:
# We define the trial and test functions.
# These are the objects that formally get
# replaced by the linear combinations of
# shape functions in order to define the
# associated matrix.
U_tria = dolfin.TrialFunction(fs)
U_test = dolfin.TestFunction(fs)

**Q4.
Complete and execute the following code**

Hints:
 * `dolfin.inner(f, g)` returns the inner (i.e., scalar) product between the two tensors `f` & `g`.
(If `f` & `g` are zero-order tensors, i.e., scalars, it is a simple product;
if they are first-order tensors, i.e., vectors, it is a dot product;
if they are second-order tensors, i.e., matrices, it is a double dot product; …)
 * `f * dolfin.dx(domain=mesh)` is the variational form corresponding to the integral of the scalar function `f` over the domain `mesh`.
 * `dolfin.assemble(form)` will assemble the variational form `form`.
(If `form` has arity 0, i.e., if it is a scalar, independent from any test or trial function, it will return its integral, i.e., a scalar;
if it has arity 1, i.e., if it is a linear form of a test function, but independent from any trial function, it will return its integral as a finite element vector;
if it has arity 2, i.e., if it is a bilinear form of a test function and a trial function, it will return its integral as a finite element matrix.)

In [ ]:
# We turn rho into a dolfin.Constant,
# so that if we change its numerical value,
# FEniCS will not regenerate and recompile
# the variational forms and associated
# assembly procedures.
rho = dolfin.Constant(rho)

# Mass variational form
M_form = ### YOUR CODE HERE ###

# Mass matrix
M = ### YOUR CODE HERE ###

We now assemble the stiffness matrix.

**Q5.
Complete and execute the following code**

Hints:
 * `dolfin.grad(f)` returns the gradient of the tensor `f`.
(If `f` is a zero-order tensor, i.e., a scalar, it is a first-order tensor, i.e., a vector;
if it is a first-order tensor, i.e., a vector, it is a second-order tensor, i.e., a matrix; …)
 * `dolfin.sym(f)` returns the symmetric part of the second order tensor `f`, i.e, the second order tensor (f + f.T)/2.
 * `dolfin.tr(f)` returns the trace of the second order tensor `f`.
 * `dolfin.Identity(d)` returns the identity second order tensor in dimension `d`.
 * `dolfin.inner(f, g)` returns the inner (i.e., scalar) product between the two tensors `f` & `g`.
(If `f` & `g` are zero-order tensors, i.e., scalars, it is a simple product;
if they are first-order tensors, i.e., vectors, it is a dot product;
if they are second-order tensors, i.e., matrices, it is a double dot product; …)
 * `f * dolfin.dx(domain=mesh)` is the variational form corresponding to the integral of the scalar function `f` over the domain `mesh`.
 * `dolfin.assemble(form)` will assemble the variational form `form`.
(If `form` has arity 0, i.e., if it is a scalar, independent from any test or trial function, it will return its integral, i.e., a scalar;
if it has arity 1, i.e., if it is a linear form of a test function, but independent from any trial function, it will return its integral as a finite element vector;
if it has arity 2, i.e., if it is a bilinear form of a test function and a trial function, it will return its integral as a finite element matrix.)

In [ ]:
# We turn lambda & mu into dolfin.Constants.
lmbda = dolfin.Constant(lmbda)
mu    = dolfin.Constant(mu)

# Stress (associated to the trial function)
eps_tria = ### YOUR CODE HERE ###
sig_tria = ### YOUR CODE HERE ###

# Strain (associated to the test function)
eps_test = ### YOUR CODE HERE ###

# Stiffness variational form
K_form = ### YOUR CODE HERE ###

# Stiffness matrix
K = ### YOUR CODE HERE ###

### Eigenvalue solver

The generalized eigenvalue problem will be solved using [SLEPc](https://slepc.upv.es), which is the *de facto* standard for this type of problems, and is integrated into FEniCS.
It can directly operate on the [PETSc](https://www.mcs.anl.gov/petsc) arrays assembled by FEniCS.

We create the solver.

In [ ]:
# (In order to create the SLEPc solver, we need to access the PETSc objects underlying the dolfin arrays…)
K_petsc = dolfin.as_backend_type(K)
M_petsc = dolfin.as_backend_type(M)
eigensolver = dolfin.SLEPcEigenSolver(K_petsc, M_petsc)

The SLEPc eigenvalue solver can handle a variety of different problems and can (needs to) be tuned properly.
For the full set of settings one can refer to [the SLEPc manual](https://slepc.upv.es/documentation/manual.htm).
Searching for all eigenvalues is practically prohibitive, so we only compute a subset.
In the present case, we are interested in the smallest (magnitude) eigenvalues, which correspond to low vibration frequencies.
However, eigenvalue solvers are designed to compute the largest eigenvalues, so that we have to tell the solver to perform an 'inversion' of the problem, which is done through the option 'shift-and-invert'—the shift in our case is $0$, which is the smallest (in magnitude) eigenvalue.

In [ ]:
# Problem type: "gen_hermitian" -> generalized hermitian, i.e.,
# generalized eigenvalues with a self-adjoint (i.e., in case of
# real numbers, symmetric) linear operators (matrices).
eigensolver.parameters["problem_type"] = "gen_hermitian"

# General transform
eigensolver.parameters["spectral_transform"] = "shift-and-invert"

# Actual shift value: here zero
eigensolver.parameters["spectral_shift"] = 0.

Now let us solve for *some* eigenvalues and eigenvectors.
*Some* because, as mentioned above, solving for eigenvalues is expensive.
In particular it becomes more expensive the further one departs from the target values, which is $0$ in our case.
Funny thing is, you never know how much modes you will have at the end!

In [ ]:
# Solve the eigenproblem, asking for a certain number of modes…
eigensolver.solve(N_modes)
print("N_modes:", N_modes)

# The number of modes actually computed…
N_pairs = eigensolver.get_number_converged()
print("N_pairs:", N_pairs)

The solver provides us with real and imaginary parts of eigenvalues and eigenvectors, so we extract the real parts, making sure the imaginary parts are small.

In [ ]:
def get_real(pair):
    eval_real, eval_imag, evec_real, evec_imag = pair
    if (eval_imag > 1e-12):
        raise RuntimeError("Imaginary part of eigenvalue is not zero (< 1e-12) but {:.16e}".format(eval_imag))
    return eval_real, evec_real

pairs = [get_real(eigensolver.get_eigenpair(k_pair)) for k_pair in range(N_pairs)]

We print the eigenvalues.

In [ ]:
print([pair[0] for pair in pairs])

**Q6.
What do you notice regarding the first six eigenvalues?
Why is that?**

**Q7.
What do you notice regarding the following eigenvalues?
Why is that?**

We save the eigenvectors for further visualization.
Note that SLEPc scales the eigenvectors such that $\mathbf{v} \cdot \mathbf{M} \cdot \mathbf{v} = 1$.

In [ ]:
# Cleaning
os.system("rm -f E9.1-Vibrations/*")

# Output file
pvd_file = dolfin.File("E9.1-Vibrations/modes.pvd")

# Function object for proper output
eigenvector = dolfin.Function(fs, name="eigenvector")

# Writing the undeformed mesh
pvd_file.write(eigenvector, 0)

# Loop over computed pairs
for pair in pairs:
    # Eigenvalue
    eigenvalue = pair[0]
    
    # Eigenvector
    eigenvector.vector().set_local(pair[1].get_local())

    # Write the eigenvector, the time stamp corresponding to the eigenvalue
    pvd_file.write(eigenvector, eigenvalue)

### Visualization

In [ ]:
viewer = lib.DisplacementViewer(pvd_folder="E9.1-Vibrations", pvd_filename="modes.pvd", state_label="frequency")
viewer.show()

### Analysis

**Q8.
What happens if you ask for (much) more modes?**

**Q9.
What is the influence of the physical parameters (notably, the thickness, the Young modulus) on the results?**

**Q10.
What is the influence of the numerical parameters (notably, the mesh size) on the results?**

## Bonus

**Q11.
Block the rigid body motion while letting the plate free.
How does that impact the results?**

**Q12.
Clamp the plate at its perimeter.
How does that impact the results?**